In [2]:
%pip install web3

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------- ----------------- 0.8/1.4 MB 5.9 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 6.0 MB/s  0:00:00
   ---------------------------------------- 0.0/587.5 kB ? eta -:--:--
   ---------------------------------------- 587.5/587.5 kB 8.6 MB/s  0:00:00
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 1.8/1.8 MB 10.0 MB/s  0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ------------------------------------ --- 1.8/2.0 MB 9.8 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 9.7 MB/s  0:00:00
   ---------------------------------------- 0.0/946.5 kB ? eta -:--:--
   ---------------------------------------- 946.5/946.5 kB 9.2 MB/s  0:00:00
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   ------- -------------------------------- 1.8/9.5 MB 9

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from web3 import Web3
import json

In [9]:
rpc_url = " http://127.0.0.1:8545/"
web3 = Web3(Web3.HTTPProvider(rpc_url))

if not web3.is_connected():
    raise Exception("Couldn't connect to Hardhat node")
else:
    print("Connected to Hardhat node")

SENDER_PRIVATE_KEY = "0xac0974bec39a17e36ba4a6b4d238ff944bacb478cbed5efcae784d7bf4f2ff80"
SENDER_ADDRESS = web3.eth.account.from_key(SENDER_PRIVATE_KEY).address

RECEIVER_1_ADDRESS = "0x70997970C51812dc3A010C7d01b50e0d17dc79C8"
RECEIVER_2_ADDRESS = "0x3C44CdDdB6a900fa2b585dd299e03d12FA4293BC"

CONTRACT_ADDRESS = "0x5FbDB2315678afecb367f032d93F642f64180aa3"

print(f"Sender address: {SENDER_ADDRESS}")
print(f"Contract address: {CONTRACT_ADDRESS}")

Connected to Hardhat node
Sender address: 0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266
Contract address: 0x5FbDB2315678afecb367f032d93F642f64180aa3


In [12]:
artifact_path = 'artifacts/contracts/AwwToken.sol/AwwToken.json'
with open(artifact_path) as artifact_file:
    artifact = json.load(artifact_file)
    CONTRACT_ABI = artifact["abi"]

token_contract = web3.eth.contract(
    address=CONTRACT_ADDRESS,
    abi=CONTRACT_ABI
)

print("Abi loaded successfully")

Abi loaded successfully


In [18]:
def execute_analyze_transaction(receiver_address, amount_in_tokens):
    print(f"Sending {amount_in_tokens} tokens to {receiver_address}")

    amount_in_wei = web3.to_wei(str(amount_in_tokens), "ether")

    transaction = token_contract.functions.transfer(receiver_address, amount_in_wei).build_transaction({
        'from': SENDER_ADDRESS,
        'nonce': web3.eth.get_transaction_count(SENDER_ADDRESS),
        'gasPrice': web3.eth.gas_price
    })

    signed_transaction = web3.eth.account.sign_transaction(transaction, private_key=SENDER_PRIVATE_KEY)

    transaction_hash = web3.eth.send_raw_transaction(signed_transaction.raw_transaction)
    print(f"Transaction hash: {web3.to_hex(transaction_hash)}")

    transaction_receipt = web3.eth.wait_for_transaction_receipt(transaction_hash=transaction_hash)
    print(f"Transaction received")
    
    gas_consumed = transaction_receipt["gasUsed"]
    print(f"Gas used: {gas_consumed}")

    is_successfull = transaction_receipt["status"]
    print(f"Transaction {"successfull" if is_successfull else "failed"}")
    print("\n=================================================\n")


execute_analyze_transaction(RECEIVER_1_ADDRESS, 20)
execute_analyze_transaction(RECEIVER_2_ADDRESS, 50)

Sending 20 tokens to 0x70997970C51812dc3A010C7d01b50e0d17dc79C8
Transaction hash: 0xb24ff892753684a3d5660a3b187426dcf0ff4d1eee48f3a5a8a17fbb1cd50577
Transaction received
Gas used: 35100
Transaction successfull


Sending 50 tokens to 0x3C44CdDdB6a900fa2b585dd299e03d12FA4293BC
Transaction hash: 0xe14dc15b621b768c1cffafbcd4f90448f090e05d082ac518d5c389d9dc7ee7c2
Transaction received
Gas used: 35088
Transaction successfull




After compliting some simple transactions we can look in hardhat node terminal and see that our estimation of used gas is equal to value listed in logs:
```
eth_sendRawTransaction
  Contract call:       AwwToken#transfer
  Transaction:         0x0aa5cc51a2a1a98d65f681b60ba7408c68962288e73ff7f4caf9710d1bb549e4
  From:                0xf39fd6e51aad88f6f4ce6ab8827279cfffb92266
  To:                  0x5fbdb2315678afecb367f032d93f642f64180aa3
  Value:               0 ETH
  Gas used:            35100 of 35100
  Block #4:            0x0af5cac4560a59e6fa5c70ae9532fb49b731ede86c21b8113dc3228b8064abef

eth_getTransactionReceipt
eth_getTransactionCount
eth_gasPrice
eth_chainId
eth_estimateGas
eth_chainId (2)
eth_sendRawTransaction
  Contract call:       AwwToken#transfer
  Transaction:         0x195e266a7c9e658b2719980b122a67469bfc2a23e23c5e440a2e1131f3fd6213
  From:                0xf39fd6e51aad88f6f4ce6ab8827279cfffb92266
  To:                  0x5fbdb2315678afecb367f032d93f642f64180aa3
  Value:               0 ETH
  Gas used:            35088 of 35088
  Block #5:            0x9529e77802e3a29e1ca0e6a0ecc8c776fcf729a0847028c885c88c09922aea0c
```